# Determine all coefficinets of each possible pairing

In [175]:
from toroidalFilament_dir.geometry_TT1 import coil_angle_dict as angle_dict
from toroidalFilament_dir.plasma_shift import cal_alpha, cal_beta
from toroidalFilament_dir.DxDz import cal_newton_DxDz as cal_DxDz
from bintrees import AVLTree
import numpy as np
from tqdm import tqdm
import pickle

In [69]:
pair_names = [[i,i-6] for i in range(11,13)] + [[i,i+6] for i in range(1,5)] #names of probes in each crosses
pair_angles = [[angle_dict[i], angle_dict[i-6]] for i in range(11,13)] + [[angle_dict[i], angle_dict[i+6]] for i in range(1,5)] #angle of probes in each cross

#Determine all possible probe arrays
def count_all_arrays(lst):
    #find all possible arrays
    all_arrays = []
    for i in range(len(lst)):
        for j in range(i+1,len(lst)):

            i_pair, j_pair = lst[i], lst[j]

            new_array = i_pair + j_pair
            if new_array not in all_arrays: all_arrays.append(new_array)

    #lst stores data in terms of opposing probes, rearrange in terms of counter-clockwise
    for i in range(len(all_arrays)):
        temp = all_arrays[i][1]
        all_arrays[i][1] = all_arrays[i][2]
        all_arrays[i][2] = temp

    return all_arrays

all_arrays_names = count_all_arrays(pair_names)
all_arrays_angles = count_all_arrays(pair_angles)

In [122]:
def probe_lst_to_str(lst):
    arr_str = ""
    for i, probe_num in enumerate(arr_name):
        if i == 0:
            arr_str += str(probe_num)
        else: arr_str += " " + str(probe_num)
    return arr_str

In [70]:
print(f"""
number of arrays: {len(all_arrays)}
all possible arrays: {all_arrays_names}
""")


number of arrays: 15
all possible arrays: [[11, 12, 5, 6], [11, 1, 5, 7], [11, 2, 5, 8], [11, 3, 5, 9], [11, 4, 5, 10], [12, 1, 6, 7], [12, 2, 6, 8], [12, 3, 6, 9], [12, 4, 6, 10], [1, 2, 7, 8], [1, 3, 7, 9], [1, 4, 7, 10], [2, 3, 8, 9], [2, 4, 8, 10], [3, 4, 9, 10]]



# Store coefficients in a dictionary of trees

In [125]:
# create dictionary to store AVLtrees
alpha_dict = dict()
for arr_name in all_arrays_names:
    arr_str = probe_lst_to_str(arr_name)
    alpha_dict[arr_str] = None

beta_dict = alpha_dict.copy()

In [153]:
#find calculate coefficients for each probes

def mk_shift_tree(lst_angles:list[float],shift_lst:list[float],shift_domain:list[float],coeff_callable:callable,taylor_order:int):
    """
    calculate coefficients and covariance of shift
    Args:
        lst_angles: angles of each probes in radian
        shift_lst: value of shift in the direction of coeff_callable
        shift_domain: domain of curve to be fit
        coeff_callable: function to calculate coefficients alpha/beta
        taylor_order: order of taylor expansion

    Returns: tree of coefficients
    """
    coeff_tree = AVLTree()
    for shift_amount in shift_lst:
        shift_coeff_cov = coeff_callable(cal_DxDz,0,taylor_order,shift_amount,lst_angles,shift_domain)
        coeff_tree.insert(shift_amount,shift_coeff_cov)
    
    return coeff_tree

shift_domain = np.arange(-0.13,0.13,0.001) #all shift values by 1 mm
for i in tqdm(range(len(all_arrays_names))):
    #determine name of array and convert to string
    arr_name = all_arrays_names[i]
    arr_str = probe_lst_to_str(arr_name)
    
    #retreive the relative angle of each probe
    arr_angles = all_arrays_angles[i]
    
    #create tree for alpha and beta
    alpha_tree = mk_shift_tree(arr_angles,shift_lst = shift_domain,shift_domain=shift_domain,coeff_callable=cal_alpha,taylor_order=3)
    beta_tree = mk_shift_tree(arr_angles,shift_lst = shift_domain,shift_domain=shift_domain,coeff_callable=cal_beta,taylor_order=3)

    alpha_dict[arr_str] = alpha_tree
    beta_dict[arr_str] = beta_tree


100%|██████████| 15/15 [01:53<00:00,  7.57s/it]


In [155]:
alpha_dict["11 2 5 8"][-0.13]

(array([ -0.02350816,  -1.2191999 ,  -1.3791194 , -17.84154794]),
 array([[ 1.95002830e-09,  1.15998453e-08, -5.04856684e-07,
         -4.03892010e-06],
        [ 1.15998453e-08,  1.04862622e-06, -6.59757785e-06,
         -1.58797903e-04],
        [-5.04856684e-07, -6.59757785e-06,  2.28174017e-04,
          2.27297329e-03],
        [-4.03892010e-06, -1.58797903e-04,  2.27297329e-03,
          3.58127975e-02]]))

In [172]:
def find_nearest(num,tree):
    """
    find value of nearest key neighbor of a given tree
    Args:
        num: input number to find nearest key's value number
        tree: tree to be traversed

    Returns: value of the nearest key
    """
    tree.insert(num,num)

    if tree.max_key() == num:
        result = tree.prev_item(num)[1]
        tree.remove(num)
        return result

    elif tree.min_key() == num:
        result = tree.succ_item(num)[1]
        tree.remove(num)
        return result

    (left_key,left_value), (right_key,right_value) = tree.prev_item(num), tree.succ_item(num)
    tree.remove(num)

    min_diff = min(abs(left_key - num), (abs(right_key - num)))
    if min_diff == left_key:
        return left_key

    return right_value

In [174]:
find_nearest(100,alpha_dict["11 2 5 8"])

(array([-0.01173408, -0.6683962 ,  0.93316647, -4.25094953]),
 array([[ 2.84899442e-08, -3.92574277e-08, -1.70251228e-06,
          4.07332927e-06],
        [-3.92574277e-08,  7.35566618e-06,  1.15058431e-05,
         -3.52763635e-04],
        [-1.70251228e-06,  1.15058431e-05,  1.78296678e-04,
         -8.08999486e-04],
        [ 4.07332927e-06, -3.52763635e-04, -8.08999486e-04,
          2.00179172e-02]]))

In [180]:
with open("shift_coefficients.pkl","wb") as f:
    pickle.dump(alpha_dict,f)
    pickle.dump(beta_dict,f)